# Тестовый ноутбук<br>

<div><span style="font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;, &quot;Segoe UI Symbol&quot;;"><br></span></div><div><span style="font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;, &quot;Segoe UI Symbol&quot;;">Попробуйте исполнить ячейки и убедитесь, что spark поднимается и работает.</span></div><div><span style="font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;, &quot;Segoe UI Symbol&quot;;">Если вы будете создавать свой ноутбук для ДЗ, скопируйте конфигурацию из этого ноутбука</span></div>


In [1]:
val sc = spark.sparkContext

In [2]:
// Функция для расчета числа Pi методом Монте-Карло

val count = sc.parallelize(1 to 100000).filter { _ =>
  val x = math.random
  val y = math.random
  x*x + y*y < 1
}.count()
println(s"Pi is roughly ${4.0 * count / 100000}")

Pi is roughly 3.14288


In [3]:
val df = spark.read.option("header", "true").csv("housing.csv")
df.show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|          341300.0|       NEAR BAY|
|  -122.25|   37.85|              

In [4]:
val df = spark.read.option("header", "true").csv("datasets/ppkmSentiment/ppkm_dataset.csv")
df.show()


+-------+--------------------+
|  class|             comment|
+-------+--------------------+
|positif|Kami siap laksana...|
|positif|Siap melaksanakan...|
|positif|Siap dukung dan s...|
|positif|Langkah 3M ini su...|
|positif|Siap amankan selu...|
|positif|Siap utk di sosia...|
|positif|Mendukung kebijak...|
|positif|Mari bersama cega...|
|positif|Mari kita suksesk...|
|positif|Siap kawal dan am...|
|positif|Semoga pak gubern...|
|positif|Semoga bapak sela...|
|positif|     Semangat selalu|
|positif|Semoga berhasil.....|
|positif|patuhi prokes dan...|
|positif|terimakasih atas ...|
|positif|Sehat selalu pemi...|
|positif|Semoga selalu dib...|
|positif|semoga pandemi se...|
|positif|Makasih bu, sukse...|
+-------+--------------------+
only showing top 20 rows



In [5]:
/*
Плюсы MSJ:
    1. Если данные не умещаются в памяти, MSJ может обрабатывать большие объемы данных, используя настройки persist и caching.
    2. MSJ может быть использован для присоединения двух или более таблиц, в то время как Hash Join - для двух таблиц.
    3. В случае присоединения неупорядоченных данных, MSJ более эффективным, так как не требует преобразования данных в пары ключ-значение.
    4. Позволяет сортировать данные во время операции соединения

Недостатки MSJ:
    1. Времени для выболнения требуется больше, чем у Hash Join, так как требуется дополнительное сортировка данных перед присоединением.
    2. Время выполнения MSJ зависит от количества элементов в соединяемых списках, а также от порядка элементов в списках.
*/

val wcDf: DataFrame = df
    // разделяем комментарий по несловообразующим символам и _, который нас тоже не устраивает 
    .select(explode(split($"comment", raw"[\W_]+")).as("word"))
    .where($"word" =!= "")
    // приведение к нижнему регистру
    .withColumn("word", lower($"word"))
    .groupBy("word")
    .count()
    .orderBy($"count".desc)

val stopList: List[String] = spark.conf
    .get("spark.stopList")
    .split(" ")
    .toList

val updatedWordCountDF: DataFrame = wcDf
    //Изменение порядка колонок
    .select(wcDf("count"), wcDf("word"))
    // исключение слов из конфига спарка
    .filter(!($"word".isin(stopList: _*)))

updatedWordCountDF.show(false)

//Сохранение в формате tsv
updatedWordCountDF.write
    .mode("overwrite")
    .option("sep", "\t")
    .csv("output5.tsv")

//чтение формата tsv
 val readDF: DataFrame = spark.read.option("sep", "\t").csv("output3.tsv")
readDF.show(false)


+-----+------------+
|count|word        |
+-----+------------+
|111  |mikro       |
|88   |dan         |
|84   |co          |
|83   |t           |
|82   |https       |
|78   |di          |
|72   |ppkmmikro   |
|70   |19          |
|66   |masyarakat  |
|60   |yg          |
|44   |kegiatan    |
|43   |pembatasan  |
|43   |perpanjangan|
|34   |rt          |
|32   |ada         |
|31   |berbasis    |
|30   |yang        |
|28   |maret       |
|27   |2021        |
|27   |untuk       |
+-----+------------+
only showing top 20 rows

+---+--------------+
|_c0|_c1           |
+---+--------------+
|1  |tumbuh        |
|1  |artinya       |
|1  |tagihan       |
|1  |erwin3977     |
|1  |absurd        |
|1  |bangun        |
|1  |pembuat       |
|1  |kabar         |
|1  |gini          |
|1  |tilang        |
|1  |australia     |
|1  |dilockdown    |
|1  |kayanya       |
|1  |kesabaran     |
|1  |disekat       |
|1  |kerjaannya    |
|1  |serma         |
|1  |grob          |
|1  |lele          |
|1  |ppk

In [6]:
Какие плюсы и недостатки у Merge Sort Join в отличии от Hash Join? (1 балл)
Соберите WordCount приложение, запустите на датасете ppkm_sentiment (1 балл)
Измените WordCount так, чтобы он удалял знаки препинания и приводил все слова к единому регистру (1 балл)
Измените выход WordCount так, чтобы сортировка была по количеству повторений, а список слов был во втором столбце, а не в первом (1 балл)
Измените выход WordCount, чтобы формат соответствовал TSV (1 балл)
Добавьте в WordCount возможность через конфигурацию задать список стоп-слов, которые будут отфильтрованы во время работы приложения (2 балла)
Выполните broadcast join на двух датасетах, не используя метод join(). Можно использовать любые предварительные трансформации. В качестве исходных данных возьмите Company.csv и Company_Tweet.csv из датасета Tweets about the Top Companies from 2005 to 2020 (3 балла)

In [7]:
// HW 6
def getData(filepath: String): DataFrame = {
  spark.read
    .option("header", "true")
    .csv(filepath)
    .withColumn("label", $"label".cast("int"))
    .filter($"label" === 1 || $"label" === 0)
}

val trainDataset: DataFrame = getData("datasets/IMDBSentimentAnalysis/Train.csv")
val testDataset: DataFrame = getData("datasets/IMDBSentimentAnalysis/Test.csv")
val validDataset: DataFrame = getData("datasets/IMDBSentimentAnalysis/Valid.csv")

In [8]:
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.{Pipeline}
import org.apache.spark.ml.feature.{HashingTF, Tokenizer}
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator

val tokenizer = new Tokenizer()
    .setInputCol("text")
    .setOutputCol("words")
val hashingTF = new HashingTF()
    .setInputCol(tokenizer.getOutputCol)
    .setOutputCol("features").setNumFeatures(12000)
val lr = new LogisticRegression()
  .setMaxIter(10)
  .setRegParam(0.3)
val pipeline = new Pipeline()
    .setStages(Array(tokenizer, hashingTF, lr))

val model = pipeline.fit(trainDataset)
val predictions = model.transform(validDataset)

val evaluator = new MulticlassClassificationEvaluator()
  .setMetricName("accuracy")
  
val accuracy = evaluator.evaluate(predictions)
println(s"Accuracy = $accuracy")

Accuracy = 0.8643835616438356
